# Journey to top 8
Here we add columns to tournament_info_df that includes the top 8 players and their location in the top 8 bracket and the index of the earlier sets they played in the tournament. 

### Load packages


In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from tqdm import tqdm
import tqdm
import os

if os.path.exists('/workspace/data_2'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data_2/'
else:
    data_path = '../data/'

### Load Data

In [2]:
tournament_info_df = pd.read_pickle(data_path + 'tournament_info_df.pkl')
tournament_info_df.head(1)


,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,US,VA,Fall's Church,10,"[[Rishi, 1], [15634, 3], [6126, 4], [Chu, 8], ...",{},b'{}',0,NaN,NaN


In [3]:
sets_df = pd.read_pickle(data_path + 'sets_top_8_labeled_df.pkl')
sets_df.head(1)

,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data,top_8,top_8_location_names,valid_top_8_bracket,top_8_bracket_location_names,major
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,Chillin,5620,1,3,"[R1, Round 1, Round 1]",,1,A,5,[],False,,False,,False


### Filter tournaments
We filter the tournaments with a valid top 8 bracket.

In [4]:
valid_bracket_keys = list(sets_df[sets_df['valid_top_8_bracket']==True]['tournament_key'].unique())
print(f"{len(valid_bracket_keys)/tournament_info_df.shape[0]:.1%} of tournaments have a valid top 8 bracket.")

valid_sets_df = sets_df[sets_df['tournament_key'].isin(valid_bracket_keys)].copy()
valid_tournament_info_df = tournament_info_df[tournament_info_df['key'].isin(valid_bracket_keys)].copy()

76.8% of tournaments have a valid top 8 bracket.


We add columns to ``tournament_info_df`` that contain the players in the top 8 and what their position in the bracket was.

In [5]:
def add_top_8_players(valid_sets_df=valid_sets_df, valid_tournament_info_df=valid_tournament_info_df):
    # Filter valid sets
    loc_names = ['WSF_A', 'WSF_B', 'LN_A', 'LN_B']
    masked_df = valid_sets_df[
        (valid_sets_df['valid_top_8_bracket'] == True) &
        (valid_sets_df['tournament_key'].isin(valid_bracket_keys)) &
        (valid_sets_df['top_8_bracket_location_names'].isin(loc_names))    
    ].copy()
    
    # Add 'key' column
    masked_df['key'] = masked_df['tournament_key']
    
    # Melt 'p1_id' and 'p2_id' into long format
    melted_df = masked_df.melt(
        id_vars=['key', 'top_8_bracket_location_names'],
        value_vars=['p1_id', 'p2_id'],
        var_name='pn_id',
        value_name='player_id'
    )
    # Create 'loc_id' column
    melted_df['loc_id'] = melted_df['top_8_bracket_location_names'] + '_' + melted_df['pn_id'].apply(lambda x: x.split('_')[0])
    
    # Pivot the table
    pivot_df = melted_df.pivot_table(
        index='key',
        columns='loc_id',
        values='player_id',
        aggfunc='first'
    ).reset_index()
    
    # Merge with tournament info
    df = valid_tournament_info_df.copy()
    df = df.merge(pivot_df, on='key', how='left')
    
    return df

top_8_tournament_df = add_top_8_players()
top_8_tournament_df.head(2)
    

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,...,lat,lng,LN_A_p1,LN_A_p2,LN_B_p1,LN_B_p2,WSF_A_p1,WSF_A_p2,WSF_B_p1,WSF_B_p2
0,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,...,NaN,NaN,math,lain,tm,Bbbbbbbbbexic,kjh,Mew2king (unpaid),Ginger,1008
1,melee,httpsparagonchallongecomla_2015_melee_singles,Paragon Los Angeles 2015,challonge,https://paragon.challonge.com/la_2015_melee_si...,,15,,2015-09-06 23:45:46,2015-09-07 20:33:07,...,NaN,NaN,1008,1023,13932,1004,1000,16342,1003,4465


Now we add columns to ``tournament_info_df`` that contain the 

In [9]:
def add_top_8_previous_sets(valid_sets_df=valid_sets_df, top_8_tournament_df=top_8_tournament_df):
    
    # Copy tournament dataframe
    valid_tourn_df = top_8_tournament_df.copy()
    
    # Make a list of the columns containing the top 8 players
    player_cols = ['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']
        
    for player_col in tqdm.tqdm(player_cols):
        
        # Add a losers_id column
        valid_df = valid_sets_df.copy()
        valid_df['loser_id'] = valid_df['p1_id']
        mask = valid_df['winner_id'] == valid_df['p1_id']
        valid_df.loc[mask, 'loser_id'] = valid_df['p2_id']

        # Add a column to valid_sets_copy that contains the player we are interested in.
        col_to_merge = ['key', player_col]
        merge_df = pd.merge(left=valid_df, right=top_8_tournament_df[col_to_merge], left_on='tournament_key', right_on='key')
        
        # Add a column to merge_df that says if the relevant player played in that set.
        merge_df['top_8_player_set'] = (merge_df['winner_id'] == merge_df[player_col]) | (merge_df['loser_id'] == merge_df[player_col])
        
        # Filter the sets to the ones where the player we are interested in played outside the top 8
        merge_df = merge_df[merge_df['top_8_player_set'] & (merge_df['valid_top_8_bracket'] == False)]

        # Add a column to merge_df that says if the relevant player won that set.
        merge_df['top_8_player_wins'] = (merge_df['winner_id'] == merge_df[player_col])

        # Add a column to merge_df that contains the index of the set and if the player we are interested in won.
        merge_df['top_8_player_matches'] = list(zip(merge_df.index, merge_df['top_8_player_wins']))

        dfs = []

        # Iterate through the tournaments using groupby object and collect the column top_8_player_matches into `dfs`
        for i, (key, group) in enumerate(merge_df.groupby('tournament_key')):
            dfs.append([key, list(group['top_8_player_matches'])])

        # Create the DataFrame with the corrected `data` parameter
        df = pd.DataFrame(dfs, columns=['key', (player_col + '_non_top_8_sets')])

        valid_tourn_df = pd.merge(valid_tourn_df, df, on='key', how='left')
    
    return valid_tourn_df


top_8_tournament_with_sets_df = add_top_8_previous_sets()
top_8_tournament_with_sets_df.iloc[:3,-16:].head()

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:15<00:00,  1.89s/it]


,LN_A_p1,LN_A_p2,LN_B_p1,LN_B_p2,WSF_A_p1,WSF_A_p2,WSF_B_p1,WSF_B_p2,LN_A_p1_non_top_8_sets,LN_A_p2_non_top_8_sets,LN_B_p1_non_top_8_sets,LN_B_p2_non_top_8_sets,WSF_A_p1_non_top_8_sets,WSF_A_p2_non_top_8_sets,WSF_B_p1_non_top_8_sets,WSF_B_p2_non_top_8_sets
0,math,lain,tm,Bbbbbbbbbexic,kjh,Mew2king (unpaid),Ginger,1008,"[(39, True), (65, True), (78, False), (165, Tr...","[(32, True), (62, True), (77, False), (164, Tr...","[(40, True), (66, True), (79, True), (85, Fals...","[(47, True), (69, True), (80, False), (159, Tr...","[(36, True), (64, True), (78, True), (85, True)]","[(28, True), (60, True), (76, True), (84, True)]","[(52, True), (72, True), (82, True), (87, True)]","[(44, True), (68, True), (80, True), (86, True)]"
1,1008,1023,13932,1004,1000,16342,1003,4465,"[(210, True), (228, True), (237, True), (241, ...","[(193, True), (219, True), (232, False), (286,...","[(188, True), (217, False), (273, True), (283,...","[(190, True), (218, False), (272, True), (282,...","[(182, True), (214, True), (230, True), (238, ...","[(194, True), (220, True), (233, True), (239, ...","[(206, True), (226, True), (236, True), (241, ...","[(198, True), (222, True), (234, True), (240, ..."
2,Hack,Vdogg,4215,Thomas,6103,6189,4465,12870,"[(328, True), (350, True), (361, True), (366, ...","[(310, True), (341, True), (356, False), (414,...","[(314, True), (343, True), (357, True), (364, ...","[(320, True), (346, True), (359, True), (365, ...","[(316, True), (344, True), (358, True), (365, ...","[(308, True), (340, True), (356, True), (364, ...","[(324, True), (348, True), (360, True), (366, ...","[(332, True), (352, True), (362, True), (367, ..."


In [14]:

col_to_merge = ['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
       'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2',
       'LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
       'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
       'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
       'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets']

In [15]:
result = pd.merge(tournament_info_df, top_8_tournament_with_sets_df[col_to_merge], how='left', left_index=True, right_index=True)


In [16]:
result.to_pickle(data_path + 'top_8_tournament_previous_sets_and_results_df')

In [17]:
result.columns

Index(['game', 'key', 'cleaned_name', 'source', 'tournament_name',
       'tournament_event', 'season', 'rank', 'start', 'end', 'country',
       'state', 'city', 'entrants', 'placings', 'losses', 'bracket_types',
       'online', 'lat', 'lng', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
       'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2',
       'LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
       'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
       'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
       'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets'],
      dtype='object')